# Estadísticos AIC, BIC y Cp

In [1]:
# Bibliotecas a utilizar 
#import sys
#!{sys.executable} -m pip install RegscorePy 
import numpy                   as np      # Uso de álgebra lineal, funciones vectoriales
import pandas                  as pd      # Trabajar con DataFrames
import statsmodels.formula.api as smf     # Modelos estadísticos
import RegscorePy              as rsp     # AIC, BIC y Cp usando la descomposición de cuadrados

# Referencias::
# https://pypi.org/project/RegscorePy/
# https://virtual.uptc.edu.co/ova/estadistica/docs/libros/2007315/lecciones_html/capitulo_8/leccion1/leccion1-4/cpmallows.html  

# Datos :: mtcars

In [2]:
# Leemos los datos
datos = pd.read_csv("mtcars.csv")
datos.head()

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


# Modelo completo

In [3]:
# Generamos el modelo con todas las variables (incluyendo intercepto)
modelo_completo = smf.ols('mpg ~ 1 + cyl + disp + hp + drat + wt + qsec + vs + am + gear + carb', data = datos).fit()
print( modelo_completo.summary() )

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.869
Model:                            OLS   Adj. R-squared:                  0.807
Method:                 Least Squares   F-statistic:                     13.93
Date:                Wed, 11 Sep 2019   Prob (F-statistic):           3.79e-07
Time:                        21:47:15   Log-Likelihood:                -69.855
No. Observations:                  32   AIC:                             161.7
Df Residuals:                      21   BIC:                             177.8
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     12.3034     18.718      0.657      0.5

Recordemos las definiciones de AIC y BIC

### Por LogVerosimilitud

$$
\begin{align*}
AIC &:= -2 \cdot \text{logLik} + p \cdot 2 \\
BIC &:= -2 \cdot \text{logLik} + p \cdot \ln(n)
\end{align*}
$$


### Por Descomposición de Cuadrados
$$
\begin{align*}
AIC &:= n \cdot \ln\left( \dfrac{SCE}{n} \right) + p \cdot 2 \\
BIC &:= n \cdot \ln\left( \dfrac{SCE}{n} \right) + p \cdot \ln(n)
\end{align*}
$$

### Además de los criterios anteriores, existe el criterio Mallows Cp (dos versiones para la desc. de cuadrados)
$$
\begin{align*}
Cp := \dfrac{SCE}{\hat{\sigma}^2} - n + p \cdot 2 \\
Cp := \dfrac{SCE + 2 \cdot p \cdot \hat{\sigma}^2}{n}
\end{align*}
$$

donde $SSE$ es la Suma de Cuadrados del Error (también conocido de los residuales), $n$ es el tamaño de muestra y $p$ es el número de covariables en la matriz de diseño (incluyendo el intercepto) y **$\hat{\sigma}^2$ es el estimador insesgado de la varianza de los errores del modelo completo (con todas las covariables y el intercepto).**

In [4]:
# Calculamos el AIC y BIC
s2 = np.sum( np.array( modelo_completo.resid )**2 ) / ( 32 - 11 ) # n-p
print("AIC :: ", 32 * np.log( np.sum( np.array( modelo_completo.resid )**2 ) / 32 ) + 11 * 2 )
print("BIC :: ", 32 * np.log( np.sum( np.array( modelo_completo.resid )**2 ) / 32 ) + 11 * np.log(32) )
print(" Cp :: ", np.sum( np.array( modelo_completo.resid )**2 ) / s2 - 32 + 11 * 2 )

AIC ::  70.89774430938061
BIC ::  87.02083924017762
 Cp ::  11.0


In [5]:
print( modelo_completo.summary2() )

                 Results: Ordinary least squares
Model:              OLS              Adj. R-squared:     0.807   
Dependent Variable: mpg              AIC:                161.7098
Date:               2019-09-11 21:47 BIC:                177.8329
No. Observations:   32               Log-Likelihood:     -69.855 
Df Model:           10               F-statistic:        13.93   
Df Residuals:       21               Prob (F-statistic): 3.79e-07
R-squared:          0.869            Scale:              7.0235  
------------------------------------------------------------------
              Coef.   Std.Err.     t     P>|t|    [0.025    0.975]
------------------------------------------------------------------
Intercept    12.3034   18.7179   0.6573  0.5181  -26.6226  51.2293
cyl          -0.1114    1.0450  -0.1066  0.9161   -2.2847   2.0618
disp          0.0133    0.0179   0.7468  0.4635   -0.0238   0.0505
hp           -0.0215    0.0218  -0.9868  0.3350   -0.0668   0.0238
drat          0.7871

In [6]:
np.sum( modelo_completo.resid ** 2 ) / ( 32 - 11 )

7.023544286507174

In [7]:
modelo_completo.mse_total

36.32410282258064

In [8]:
modelo_completo.mse_resid

7.023544286507174

In [9]:
modelo_completo.mse_model

97.85527574833492

In [10]:
np.sum( np.array( modelo_completo.resid )**2 ) / modelo_completo.mse_model - 32 + 11 * 2

-8.492728890816494

In [11]:
# Calculamos algunos variables necesarias
y_real        = datos.mpg
y_estimada_mc = modelo_completo.fittedvalues
sigma2        = np.sum( np.array( modelo_completo.resid )**2 ) / ( 32 - 11 )
n = 32
p = 11

# Mostramos las estadisticas
print("AIC :: ")
print(rsp.aic.aic( y = y_real, y_pred = y_estimada_mc, p = p ))
print("")
print("BIC :: ")
print(rsp.bic.bic( y = y_real, y_pred = y_estimada_mc, p = p ))
print("")
print("Cp :: ")
print(rsp.mallow.mallow( y = y_real, y_pred = y_estimada_mc, y_sub = y_estimada_mc, k = p, p = p ))

AIC :: 
70.89774430938061

BIC :: 
87.02083924017762

Cp :: 
11.0


Notemos que el AIC y BIC que se calcula mediante estas funciones **NO** corresponden a los que calculan las bibliotecas *StatsModels* o *ScikitLearn*, sin embargo, ambas versiones son correctas. 

Las diferentes versiones son "monótonas", por lo que si en la primer versión el mejor modelo es el del AIC, entonces en la segunda versión también será el AIC (aunque sus valores no coincidan).

Para el coeficiente de Mallows es similar entre sus dos versiones.
Es decir, su valor no coincide entre una y otra versión, pero el orden si se mantiene.

Para estas tres estadisticas, un candidato a ser el mejor modelo es aquel que tenga el menor AIC / BIC / Cp.

# Otros sub-modelos

In [12]:
# Generamos el modelo de ejemplo
modelo_ejemplo = smf.ols('mpg ~ 1 + am + qsec + wt', data = datos).fit()
print( modelo_ejemplo.summary() )

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.850
Model:                            OLS   Adj. R-squared:                  0.834
Method:                 Least Squares   F-statistic:                     52.75
Date:                Wed, 11 Sep 2019   Prob (F-statistic):           1.21e-11
Time:                        21:47:15   Log-Likelihood:                -72.060
No. Observations:                  32   AIC:                             152.1
Df Residuals:                      28   BIC:                             158.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      9.6178      6.960      1.382      0.1

In [13]:
# Calculamos algunos variables necesarias 
y_real             = datos.mpg                      # Y Reales    del modelo (en este caso mpg)
y_estimada_mc      = modelo_completo.fittedvalues   # Y Ajustadas del modelo completo (todas las covariables)
y_estimada_ejemplo = modelo_ejemplo.fittedvalues    # Y Ajustadas del sub-modelo a comparar

# Mostramos las estadisticas
print("AIC :: ")
print(rsp.aic.aic( y = y_real, y_pred = y_estimada_ejemplo, p = 4 ))
print("")
print("BIC :: ")
print(rsp.bic.bic( y = y_real, y_pred = y_estimada_ejemplo, p = 4 ))
print("")
print("Cp :: ")
print(rsp.mallow.mallow( y = y_real, y_pred = y_estimada_mc, y_sub = y_estimada_ejemplo, k = 11, p = 4 ))

AIC :: 
61.30730474380213

BIC :: 
67.17024835500104

Cp :: 
0.10263573946057036


In [14]:
# Generamos el modelo de ejemplo
modelo_ejemplo = smf.ols('mpg ~ 0 + cyl + gear', data = datos).fit()
print( modelo_ejemplo.summary() )

                                 OLS Regression Results                                
Dep. Variable:                    mpg   R-squared (uncentered):                   0.942
Model:                            OLS   Adj. R-squared (uncentered):              0.938
Method:                 Least Squares   F-statistic:                              242.6
Date:                Wed, 11 Sep 2019   Prob (F-statistic):                    3.00e-19
Time:                        21:47:15   Log-Likelihood:                         -97.258
No. Observations:                  32   AIC:                                      198.5
Df Residuals:                      30   BIC:                                      201.4
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [15]:
# Calculamos algunos variables necesarias 
y_real             = datos.mpg                      # Y Reales    del modelo (en este caso mpg)
y_estimada_mc      = modelo_completo.fittedvalues   # Y Ajustadas del modelo completo (todas las covariables)
y_estimada_ejemplo = modelo_ejemplo.fittedvalues    # Y Ajustadas del sub-modelo a comparar

# Mostramos las estadisticas
print("AIC :: ")
print(rsp.aic.aic( y = y_real, y_pred = y_estimada_ejemplo, p = 3 ))
print("")
print("BIC :: ")
print(rsp.bic.bic( y = y_real, y_pred = y_estimada_ejemplo, p = 3 ))
print("")
print("Cp :: ")
print(rsp.mallow.mallow( y = y_real, y_pred = y_estimada_mc, y_sub = y_estimada_ejemplo, k = 11, p = 3 ))

AIC :: 
109.70435404459785

BIC :: 
114.10156175299703

Cp :: 
90.42286980063771


In [16]:
# Generamos el modelo de ejemplo
modelo_ejemplo = smf.ols('mpg ~ disp + hp + drat', data = datos).fit()
print( modelo_ejemplo.summary() )

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.775
Model:                            OLS   Adj. R-squared:                  0.751
Method:                 Least Squares   F-statistic:                     32.15
Date:                Wed, 11 Sep 2019   Prob (F-statistic):           3.28e-09
Time:                        21:47:15   Log-Likelihood:                -78.510
No. Observations:                  32   AIC:                             165.0
Df Residuals:                      28   BIC:                             170.9
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     19.3443      6.371      3.036      0.0

In [17]:
# Calculamos algunos variables necesarias 
y_real             = datos.mpg                      # Y Reales    del modelo (en este caso mpg)
y_estimada_mc      = modelo_completo.fittedvalues   # Y Ajustadas del modelo completo (todas las covariables)
y_estimada_ejemplo = modelo_ejemplo.fittedvalues    # Y Ajustadas del sub-modelo a comparar

# Mostramos las estadisticas
print("AIC :: ")
print(rsp.aic.aic( y = y_real, y_pred = y_estimada_ejemplo, p = 3 ))
print("")
print("BIC :: ")
print(rsp.bic.bic( y = y_real, y_pred = y_estimada_ejemplo, p = 3 ))
print("")
print("Cp :: ")
print(rsp.mallow.mallow( y = y_real, y_pred = y_estimada_mc, y_sub = y_estimada_ejemplo, k = 11, p = 3 ))

AIC :: 
72.20863520509653

BIC :: 
76.6058429134957

Cp :: 
10.070948051057343
